
This model can be used as a spam,sentiment and semantic search analyis model all in one...

In [2]:
import os, pandas as pd, numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score,f1_score,classification_report,confusion_matrix
from sklearn.neighbors import NearestNeighbors
import joblib, urllib.request, zipfile, io, tarfile


In [3]:
p=[]
for r,d,f in os.walk('/kaggle/input'):
    for g in f:
        if g.lower().endswith('.csv') and (('spam' in g.lower()) or ('sms' in g.lower())):
            p.append(os.path.join(r,g))
x=None
if len(p): x=p[0]
try:
    if x is None: raise FileNotFoundError
    df=pd.read_csv(x,encoding='latin-1')
except:
    u='https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
    b=urllib.request.urlopen(u).read()
    z=zipfile.ZipFile(io.BytesIO(b))
    t=z.read('SMSSpamCollection').decode('utf-8').splitlines()
    a=[i.split('\t',1) for i in t if '\t' in i]
    df=pd.DataFrame(a,columns=['y','x'])
else:
    if 'v1' in df.columns and 'v2' in df.columns:
        df=df[['v1','v2']].rename(columns={'v1':'y','v2':'x'})
    elif 'label' in df.columns and 'text' in df.columns:
        df=df[['label','text']].rename(columns={'label':'y','text':'x'})
    else:
        c=df.columns.tolist()
        df=df[[c[0],c[1]]].rename(columns={c[0]:'y',c[1]:'x'})
df=df.dropna()
df['y']=df['y'].astype(str).str.lower().str.strip()
df['x']=df['x'].astype(str)
df['y']=np.where(df['y'].str.contains('spam'),'spam','ham')
df.head()


,y,x
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Datasets used:
1)UCI SMS Spam Collection
2)IMDb Large Movie Review Dataset (aclImdb_v1.tar.gz)

In [4]:
q=[]
for r,d,f in os.walk('/kaggle/input'):
    for g in f:
        if g.lower().endswith('.csv') and (('imdb' in g.lower()) or ('sentiment' in g.lower()) or ('review' in g.lower())):
            q.append(os.path.join(r,g))
y=None
if len(q): y=q[0]
df2=None
try:
    if y is None: raise FileNotFoundError
    dd=pd.read_csv(y)
    c=dd.columns.tolist()
    if ('text' in dd.columns) and ('label' in dd.columns):
        df2=dd[['text','label']].rename(columns={'text':'x','label':'y'})
    elif len(c)>=2:
        df2=dd[[c[0],c[1]]].rename(columns={c[0]:'y',c[1]:'x'})
except:
    try:
        u='https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
        b=urllib.request.urlopen(u).read()
        tg=tarfile.open(fileobj=io.BytesIO(b),mode='r:gz')
        tg.extractall('/kaggle/working')
        a=[]
        for s in ['pos','neg']:
            for r,d,f in os.walk(f'/kaggle/working/aclImdb/train/{s}'):
                for g in f:
                    if g.endswith('.txt'):
                        a.append((1 if s=='pos' else 0, os.path.join(r,g)))
        np.random.seed(7)
        np.random.shuffle(a)
        a=a[:12000]
        xx=[]
        yy=[]
        for t0,t1 in a:
            try:
                with open(t1,'r',encoding='utf-8') as h:  xx.append(h.read())
            except:
                with open(t1,'r',encoding='latin-1') as h: xx.append(h.read())
            yy.append(t0)
        df2=pd.DataFrame({'x':xx,'y':yy})
    except:
        df2=None
df2.head() if df2 is not None else 'no sentiment dataset'


/tmp/ipykernel_55/877987313.py:22: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tg.extractall('/kaggle/working')


,x,y
0,*SOILER* It's fake! The whole thing is a fake!...,0
1,I thought that Mr. Dreyfuss was perfect for hi...,1
2,I'll say this much--This director is all about...,0
3,"I saw this movie, and at times, I was unnerved...",0
4,This movie was terrible. Throughout the whole ...,0


Spam model training

In [5]:
x=df['x'].values
y=(df['y'].values=='spam').astype(int)
xtr,xts,ytr,yts=train_test_split(x,y,test_size=0.2,random_state=7,stratify=y)
m=Pipeline([('t',TfidfVectorizer(lowercase=True,stop_words='english',ngram_range=(1,2),min_df=2,max_features=60000)),
            ('c',LogisticRegression(max_iter=2500))])
m.fit(xtr,ytr)
pr=m.predict(xts)
acc=accuracy_score(yts,pr)
f1=f1_score(yts,pr)
acc,f1


(0.9713004484304932, 0.8814814814814815)

Spam eval.

In [6]:
print(classification_report(yts,pr,digits=4))
print(confusion_matrix(yts,pr))


              precision    recall  f1-score   support

           0     0.9698    0.9979    0.9837       966
           1     0.9835    0.7987    0.8815       149

    accuracy                         0.9713      1115
   macro avg     0.9766    0.8983    0.9326      1115
weighted avg     0.9716    0.9713    0.9700      1115

[[964   2]
 [ 30 119]]


Sentiment model training

In [7]:
m2=None
if df2 is not None and len(df2):
    df2=df2.dropna()
    df2['x']=df2['x'].astype(str)
    df2['y']=df2['y'].astype(int)
    x2=df2['x'].values
    y2=df2['y'].values
    xtr,xts,ytr,yts=train_test_split(x2,y2,test_size=0.2,random_state=9,stratify=y2)
    m2=Pipeline([('t',TfidfVectorizer(lowercase=True,stop_words='english',ngram_range=(1,2),min_df=2,max_features=90000)),
                 ('c',LogisticRegression(max_iter=2500))])
    m2.fit(xtr,ytr)
    pr=m2.predict(xts)
    acc=accuracy_score(yts,pr)
    f1=f1_score(yts,pr)
    print(acc,f1)
    print(confusion_matrix(yts,pr))
else:
    'skipped'


0.8766666666666667 0.8783894823336073
[[1035  173]
 [ 123 1069]]


Building the semantic search index on combined texts (spam+sentiment).

In [8]:
a=df['x'].astype(str).tolist()
b=[]
if df2 is not None: b=df2['x'].astype(str).tolist()
c=a[:8000] + b[:8000]
v=TfidfVectorizer(lowercase=True,stop_words='english',ngram_range=(1,2),min_df=2,max_features=120000)
z=v.fit_transform(c)
nn=NearestNeighbors(n_neighbors=6,metric='cosine')
nn.fit(z)
len(c),z.shape


(13574, (13574, 109768))

Saving

In [9]:
joblib.dump(m,'/kaggle/working/spam.pkl')
joblib.dump(m2,'/kaggle/working/sent.pkl') if m2 is not None else None
joblib.dump(v,'/kaggle/working/vec.pkl')
joblib.dump(nn,'/kaggle/working/nn.pkl')
joblib.dump(c,'/kaggle/working/corpus.pkl')
sorted([i for i in os.listdir('/kaggle/working') if i.endswith('.pkl')])


['corpus.pkl', 'nn.pkl', 'sent.pkl', 'spam.pkl', 'vec.pkl']

testing

In [10]:
s=[
 'URGENT! You have won a free ticket, call now',
 'bro are you coming to the lab today',
 'this movie was painfully slow but the acting was great',
 'absolutely loved it, would watch again'
]
ps=m.predict_proba(s)[:,1]
pt=None
if m2 is not None: pt=m2.predict_proba(s)[:,1]
lab=[]
for i,t in enumerate(s):
    if ps[i]>0.55: lab.append('spam')
    else:
        if pt is None: lab.append('ham')
        else:
            lab.append('pos' if pt[i]>0.55 else 'neg')
list(zip(s,ps, (pt.tolist() if pt is not None else None),lab))


[('URGENT! You have won a free ticket, call now',
  np.float64(0.585037381693044),
  0.5482976895832998,
  'spam'),
 ('bro are you coming to the lab today',
  np.float64(0.056245199409015734),
  0.7404259358630905,
  'pos'),
 ('this movie was painfully slow but the acting was great',
  np.float64(0.06219929200951829),
  0.5324711551140653,
  'neg'),
 ('absolutely loved it, would watch again',
  np.float64(0.06721548851399393),
  0.8330333967434881,
  'pos')]

In [11]:
q='free prize call now'
u=v.transform([q])
dd,ii=nn.kneighbors(u,n_neighbors=6)
[(float(dd[0][k]), c[int(ii[0][k])][:180]) for k in range(len(ii[0]))]


[(0.39191776659530664, 'When you get free, call me'),
 (0.39191776659530664, 'Are you free now?can i call now?'),
 (0.39191776659530664, "Can... I'm free..."),
 (0.5763007571904433, "Sorry i'm not free..."),
 (0.5987440119991589, 'Win a £1000 cash prize or a prize worth £5000'),
 (0.5987718927257151, 'Hey now am free you can call me.')]